In [1]:
import pandas as pd

PATH = '/home/kai/data/kaggle/homecredit/'


train = pd.read_csv(PATH + 'application_train.csv')
test = pd.read_csv(PATH + 'application_test.csv')
bureau = pd.read_csv(PATH + 'bureau.csv')
bb = pd.read_csv(PATH + 'bureau_balance.csv')
b = bureau.copy()
'done'
print('shape of bureau and bb', b.shape, bb.shape)

shape of bureau and bb (1716428, 17) (27299925, 3)


# Functions

In [ ]:
def minus_name(col1, col2): return col1 + '_minus_' + col2
def minus(df, col1, col2): return df[col1] - df[col2]

def ratio_name(col1, col2): return col1 + '_divide_' + col2
def ratio(df, col1, col2): return df[col1] / (df[col2] + 1)

def positive_count(df, gp_col, col):
    group = (df[col] > 0).astype('int8')
    group = pd.concat([df[gp_col], group], axis=1).groupby(gp_col)[[col]].sum().reset_index().rename(index=str, columns={col: 'positivecount_'+'_'.join([gp_col, col])})
    return group.astype({gp_col: df[gp_col].dtype})

def count(df, gp_col, col):
    group = df[[gp_col,col]].groupby(gp_col)[[col]].count().reset_index().rename(index=str, columns={col:'count_'+gp_col})
    return group.astype({gp_col: df[gp_col].dtype})

def numerical(df, gp_col, col, agg_fun):
    _df = df.groupby(gp_col)[[col]].agg(agg_fun)
    
    columns = []
    for x in _df.columns.levels[0]:
        for y in _df.columns.levels[1]:
            columns.append('_'.join([x, y]))
    _df.columns = columns
    return _df.reset_index().astype({gp_col: df[gp_col].dtype})

def one_hot(df, gp_col, col):
    return pd.concat([df[gp_col], pd.get_dummies(df[col], prefix='onehot_' + col)], axis=1).groupby(gp_col).sum().reset_index().astype({gp_col: df[gp_col].dtype})

'done'

# bureau balance to bureau

In [ ]:
print(bb.columns)

In [ ]:

merged_col = 'SK_ID_BUREAU'

smap = {'C':0, 'X':0, '0':0, '1':1, '2':2, '3':3, '4':4, '5':5}
bb['STATUS_NUM'] = bb['STATUS'].map(smap)
# ratio
bb[ratio_name('STATUS_NUM', 'MONTHS_BALANCE')] = ratio(bb, 'STATUS_NUM', 'MONTHS_BALANCE')

# one hot sum
b = b.merge(one_hot(bb, merged_col, 'STATUS'), on=merged_col, how='left')

ignored = ['SK_ID_BUREAU', 'STATUS', 'STATUS_NUM']
for x in set(bb.columns) - set(ignored):
    print(x)
    b = b.merge(numerical(bb, merged_col, x, ['mean', 'median', 'max', 'min', 'sum']), on=merged_col, how='left')

'done'

In [ ]:
b.to_pickle(PATH + 'inter/bb2bureau.pkl')

# bureau to curr

In [6]:
import pandas as pd

PATH = '/home/kai/data/kaggle/homecredit/'

train = pd.read_csv(PATH + 'application_train.csv')
bureau = pd.read_csv(PATH + 'bureau.csv')
bureau.columns

Index(['SK_ID_CURR', 'SK_ID_BUREAU', 'CREDIT_ACTIVE', 'CREDIT_CURRENCY',
       'DAYS_CREDIT', 'CREDIT_DAY_OVERDUE', 'DAYS_CREDIT_ENDDATE',
       'DAYS_ENDDATE_FACT', 'AMT_CREDIT_MAX_OVERDUE', 'CNT_CREDIT_PROLONG',
       'AMT_CREDIT_SUM', 'AMT_CREDIT_SUM_DEBT', 'AMT_CREDIT_SUM_LIMIT',
       'AMT_CREDIT_SUM_OVERDUE', 'CREDIT_TYPE', 'DAYS_CREDIT_UPDATE',
       'AMT_ANNUITY'],
      dtype='object')

In [7]:
df = pd.concat([train['SK_ID_CURR'], test['SK_ID_CURR']], ignore_index=True).to_frame()
merged_col = 'SK_ID_CURR'

# ratio
numerator = [x for x in bureau.columns if 'AMT' in x]
denominator = [x for x in bureau.columns if 'DAY' in x] + numerator + ['CNT_CREDIT_PROLONG']
for x in numerator:
    for y in denominator:
        if (x != y) and (ratio_name(y, x) not in bureau.columns):
            print(x, y)
            bureau[ratio_name(x, y)] = ratio(bureau, x, y)
bureau[ratio_name('CNT_CREDIT_PROLONG', 'DAYS_CREDIT')] = ratio(bureau, 'CNT_CREDIT_PROLONG', 'DAYS_CREDIT')
print('ratio')

# positive count
positive_count_list = ['CREDIT_DAY_OVERDUE', 'AMT_CREDIT_MAX_OVERDUE', 'CNT_CREDIT_PROLONG', 'AMT_CREDIT_SUM_DEBT', 'AMT_CREDIT_SUM_OVERDUE']
for x in positive_count_list:
    print(x)
    df = df.merge(positive_count(bureau, merged_col, x), on=merged_col, how='left')

# one hot encoding sum
for x in ['CREDIT_ACTIVE', 'CREDIT_TYPE']:
    print(x)
    df = df.merge(one_hot(bureau, merged_col, x), on=merged_col, how='left')

# count
df = df.merge(count(bureau, merged_col, 'SK_ID_BUREAU'), on=merged_col, how='left')
count_name = [x for x in df.columns if 'count_' in x][0]

# categorical ratio
for x in df.columns:
    if 'positivecount_' in x or 'onehot_' in x:
        print(x)
        df[ratio_name(x, count_name)] = ratio(df, x, count_name)

# merge bureau balance
bb = pd.read_pickle(PATH + 'inter/bb2bureau.pkl')
bureau = bureau.merge(bb, on='SK_ID_BUREAU', how='left')

# numerical
ignored = ['SK_ID_CURR', 'SK_ID_BUREAU', 'CREDIT_ACTIVE', 'CREDIT_CURRENCY', 'CREDIT_TYPE']
for x in set(bureau.columns) - set(ignored):
    print(x)
    df = df.merge(numerical(bureau, merged_col, x, ['mean', 'median', 'max', 'min', 'sum']), on=merged_col, how='left')

columns = []
for x in df.columns:
    tmp = 'bureau_' + x if x != merged_col else x
    columns.append(tmp)
df.columns = columns
'done'

AMT_CREDIT_MAX_OVERDUE DAYS_CREDIT
AMT_CREDIT_MAX_OVERDUE CREDIT_DAY_OVERDUE
AMT_CREDIT_MAX_OVERDUE DAYS_CREDIT_ENDDATE
AMT_CREDIT_MAX_OVERDUE DAYS_ENDDATE_FACT
AMT_CREDIT_MAX_OVERDUE DAYS_CREDIT_UPDATE
AMT_CREDIT_MAX_OVERDUE AMT_CREDIT_SUM
AMT_CREDIT_MAX_OVERDUE AMT_CREDIT_SUM_DEBT
AMT_CREDIT_MAX_OVERDUE AMT_CREDIT_SUM_LIMIT
AMT_CREDIT_MAX_OVERDUE AMT_CREDIT_SUM_OVERDUE
AMT_CREDIT_MAX_OVERDUE AMT_ANNUITY
AMT_CREDIT_MAX_OVERDUE CNT_CREDIT_PROLONG
AMT_CREDIT_SUM DAYS_CREDIT
AMT_CREDIT_SUM CREDIT_DAY_OVERDUE
AMT_CREDIT_SUM DAYS_CREDIT_ENDDATE
AMT_CREDIT_SUM DAYS_ENDDATE_FACT
AMT_CREDIT_SUM DAYS_CREDIT_UPDATE
AMT_CREDIT_SUM AMT_CREDIT_SUM_DEBT
AMT_CREDIT_SUM AMT_CREDIT_SUM_LIMIT
AMT_CREDIT_SUM AMT_CREDIT_SUM_OVERDUE
AMT_CREDIT_SUM AMT_ANNUITY
AMT_CREDIT_SUM CNT_CREDIT_PROLONG
AMT_CREDIT_SUM_DEBT DAYS_CREDIT
AMT_CREDIT_SUM_DEBT CREDIT_DAY_OVERDUE
AMT_CREDIT_SUM_DEBT DAYS_CREDIT_ENDDATE
AMT_CREDIT_SUM_DEBT DAYS_ENDDATE_FACT
AMT_CREDIT_SUM_DEBT DAYS_CREDIT_UPDATE
AMT_CREDIT_SUM_DEBT AMT_CRED

'done'

## merge

In [10]:
print(df.columns)
df.shape

Index(['SK_ID_CURR', 'bureau_positivecount_SK_ID_CURR_CREDIT_DAY_OVERDUE',
       'bureau_positivecount_SK_ID_CURR_AMT_CREDIT_MAX_OVERDUE',
       'bureau_positivecount_SK_ID_CURR_CNT_CREDIT_PROLONG',
       'bureau_positivecount_SK_ID_CURR_AMT_CREDIT_SUM_DEBT',
       'bureau_positivecount_SK_ID_CURR_AMT_CREDIT_SUM_OVERDUE',
       'bureau_onehot_CREDIT_ACTIVE_Active',
       'bureau_onehot_CREDIT_ACTIVE_Bad debt',
       'bureau_onehot_CREDIT_ACTIVE_Closed',
       'bureau_onehot_CREDIT_ACTIVE_Sold',
       ...
       'bureau_AMT_CREDIT_SUM_OVERDUE_divide_DAYS_CREDIT_ENDDATE_mean',
       'bureau_AMT_CREDIT_SUM_OVERDUE_divide_DAYS_CREDIT_ENDDATE_median',
       'bureau_AMT_CREDIT_SUM_OVERDUE_divide_DAYS_CREDIT_ENDDATE_max',
       'bureau_AMT_CREDIT_SUM_OVERDUE_divide_DAYS_CREDIT_ENDDATE_min',
       'bureau_AMT_CREDIT_SUM_OVERDUE_divide_DAYS_CREDIT_ENDDATE_sum',
       'bureau_onehot_STATUS_3_mean', 'bureau_onehot_STATUS_3_median',
       'bureau_onehot_STATUS_3_max', 'bureau_onehot

(356255, 460)

In [11]:
df.to_pickle(PATH + 'inter/bureau2curr.pkl')
'done'

'done'